<a href="https://colab.research.google.com/github/chelseama0715/Yuchen-Ma/blob/main/Heston_model(Ordinary_RE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Option pricing in the (rough) Heston model（fractional Riccati equation）



In [1]:
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
 

In the rough Heston model, the pair (S,V)of the stock price and its instantaneous variance has the dynamics $\mathrm{d}S_t=S_t\sqrt{V_t}\mathrm{d}W_t$ , $S_0=s_0\in R_+$
$V_t=V_0+\frac{1}{\Gamma(\alpha)}\int_{0}^{t}(t-s)^{\alpha-1}\eta(m-V_s)\mathrm{d}s+\frac{1}{\Gamma(\alpha)}\int_{0}^{t}(t-s)^{\alpha-1}\zeta\sqrt{V_s}\mathrm{d}B_s , V_0\in R_+$
where $\eta,m,\zeta$ are positive real numbers, and the correlation between the two Brownian motion W and B is $\rho \in (-1,1)$. The parameter $\alpha \in (0,2)$ plays an important role in representing the Hurst parameter. 

 First, consider the case where the model is not rough(**$\alpha$ = 1**),
 then the equation will be reduced to $\psi^{'} = \lambda\psi^2+\mu\psi+\nu$

We get the solution: $\psi(t) = \sum a_kt^k$ with $a_0=0,a_1=\nu/\Gamma(2),a_{k+1}=(\lambda a_k^{*2}+\mu a_k)/(k+1)$


# inputs:  
$\lambda - \text{second order coefficient of the equation} \\ 
\mu - \text{first order coefficient of the equation} \\
\nu - \text{the intercept of the equation}\\
t - \text{time}$

# output:

**Solutions of the equation: $\psi(t)$**



In [54]:
lambda_1 = np.random.uniform(0.1,10,[1,400])
mu = np.random.uniform(-200,-150,[1,400])
nu = np.random.uniform(180,200,[1,400])

t = np.random.uniform(0,1/252)


r_max = 150

def return_all_the_coefficients():  # return the list of coefficients up to a_k

  coefficients = [np.nan]*(r_max+1)

  a0 = 0
  a1 = nu/math.gamma(2)
  coefficients[0] = a0
  coefficients[1] = a1

  def second_order_coefficients_convol(list_of_coefficients,k): #return a^*2_k given the first k-1 coefficients
    if k==1:
      return 0
    else:
      sum = 0
      for i in range(1,k):
        a_1 = list_of_coefficients[1]
        a_k_1 = list_of_coefficients[k-1]
        sum += a_1*a_k_1
      return sum

  def second_order_coefficients(list_of_coefficients,n):
    k = n-1
    a_k_star_quadratic = second_order_coefficients_convol(list_of_coefficients,k)
    a_k = list_of_coefficients[k]
    a_n = (lambda_1*a_k_star_quadratic + mu*a_k)/(k+1)

    return a_n

  for i in range(2,r_max+1):
    coefficients[i] = second_order_coefficients(coefficients,i)

    return coefficients
  
 

  
  

In [75]:
 coeff=return_all_the_coefficients()
 coeff

[0, array([[186.56275462, 181.67871771, 180.10193423, 180.01506436,
         195.65919222, 197.3465503 , 193.88559214, 182.06623318,
         191.4270046 , 192.76104279, 187.50871088, 195.7198527 ,
         180.55826319, 189.04934606, 187.82535983, 186.91029228,
         196.67107604, 199.31150437, 196.46738304, 183.97793647,
         196.91274393, 194.29757814, 192.90013149, 187.24208427,
         197.17735725, 181.73848843, 190.35866088, 189.2455336 ,
         195.94747359, 190.6794379 , 184.08366494, 198.27952642,
         185.40399592, 197.68974898, 183.23465232, 194.77094578,
         184.00333985, 190.96906003, 182.68639314, 182.37584117,
         196.74407027, 197.39945567, 186.89815743, 189.08726823,
         194.02911764, 183.67148514, 194.49057747, 195.58454455,
         199.37005372, 195.26922948, 180.63184611, 182.68164617,
         196.66864162, 184.63858573, 194.63764395, 194.02496756,
         190.06719094, 192.48833859, 181.31538713, 190.5052888 ,
         186.13388709,

In [48]:
len(coeff)


list

In [68]:

coef_df = pd.DataFrame(coeff)
coef_df = coef_df.rename(columns={0:"val"})
coef_df.shape[1]
coef_df

,val
0,0
1,"[[186.56275462232588, 181.67871771106817, 180...."
2,"[[-15670.747783007882, -18133.036035200497, -1..."
3,NaN
4,NaN
...,...
146,NaN
147,NaN
148,NaN
149,NaN


In [74]:
import sys

psi_t = np.mat(coeff)*t

def psi():
  global psi_t
  psi_t += np.mat(coeff)*pow(t,k)
  return psi_t

output_psi_t = psi()
output_psi_t  


/usr/local/lib/python3.7/dist-packages/numpy/matrixlib/defmatrix.py:145: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr = N.array(data, dtype=dtype, copy=copy)


matrix([[0.0,
         array([[0.13499229, 0.13145832, 0.13031739, 0.13025454, 0.14157425,
        0.14279518, 0.14029091, 0.13173871, 0.13851194, 0.13947722,
        0.13567676, 0.14161814, 0.13064758, 0.13679152, 0.13590588,
        0.13524376, 0.14230642, 0.14421697, 0.14215904, 0.13312198,
        0.14248129, 0.14058902, 0.13957786, 0.13548383, 0.14267276,
        0.13150156, 0.13773891, 0.13693348, 0.14178284, 0.13797102,
        0.13319848, 0.14347026, 0.13415384, 0.14304351, 0.13258415,
        0.14093154, 0.13314036, 0.13818058, 0.13218745, 0.13196274,
        0.14235924, 0.14283346, 0.13523498, 0.13681896, 0.14039477,
        0.13290023, 0.14072867, 0.14152024, 0.14425934, 0.14129208,
        0.13070083, 0.13218401, 0.14230466, 0.1336    , 0.14083508,
        0.14039176, 0.13752801, 0.1392799 , 0.13119542, 0.13784501,
        0.13468197, 0.14194393, 0.13135608, 0.13074752, 0.1326558 ,
        0.1427692 , 0.13617202, 0.13861712, 0.1352164 , 0.13259113,
        0.14286543, 0.137

# **Create the Neural Network**

In [ ]:
import keras
from keras.layers import Activation
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
keras.backend.set_floatx('float64')

input1 = keras.layers.Input(shape=(4,))
x1 = keras.layers.Dense(30,activation = 'elu')(input1)
x2=keras.layers.Dense(30,activation = 'elu')(x1) 
x3=keras.layers.Dense(30,activation = 'elu')(x2) 


x4=keras.layers.Dense(88,activation = 'linear')(x3)


modelGEN = keras.models.Model(inputs=input1, outputs=x4)
modelGEN.summary()

# **Fit the neural network**

In [ ]:

modelGEN.complie(loss= "mse", optimizer = "adam")

modelGEN.fit(psi_train, x_train, batch_size=32, validation_data = (psi_test,
x_test),epochs = 200, verbose = True, shuffle = 1)                                     